# Notebook 05 — Time Series Forecasting with Conformal Intervals

**Objective**: Forecast portfolio-level default rates and loan volumes using the Nixtla ecosystem (statsforecast + mlforecast) with **distribution-free conformal prediction intervals**.

**Why time series matters for credit risk**:
- IFRS 9 / CECL mandates **forward-looking** expected credit loss estimates
- Forecast intervals serve as **data-driven stress scenarios** (upper bound of 95% CI)
- Portfolio-level default rate trends complement loan-level PD models (NB03-04)
- Conformal intervals provide **finite-sample coverage guarantees**, unlike parametric ARIMA intervals

**Key components**:
1. Statistical baselines: AutoARIMA, AutoETS, AutoTheta, SeasonalNaive
2. ML forecaster: LightGBM with lag features + conformal prediction intervals
3. Expanding-window cross-validation for model comparison
4. Grade-level hierarchical forecasting with reconciliation
5. Connection to IFRS 9 provisioning and stress testing

In [ ]:
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import pandas as pd
import seaborn as sns
from lightgbm import LGBMRegressor
from loguru import logger
from mlforecast import MLForecast
from mlforecast.lag_transforms import (
    ExponentiallyWeightedMean,
    RollingMean,
    RollingStd,
)
from mlforecast.utils import PredictionIntervals

# Nixtla ecosystem
from statsforecast import StatsForecast
from statsforecast.models import (
    AutoARIMA,
    AutoETS,
    AutoTheta,
    SeasonalNaive,
)
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import mae, rmse
from utilsforecast.plotting import plot_series

# Plotting defaults
plt.rcParams.update(
    {
        "figure.figsize": (14, 5),
        "axes.grid": True,
        "grid.alpha": 0.3,
        "font.size": 11,
    }
)
sns.set_palette("colorblind")

# Paths
DATA_DIR = Path("../data/processed")
MODEL_DIR = Path("../models")
MODEL_DIR.mkdir(exist_ok=True)

HORIZON = 12  # 12-month forecast (IFRS 9 Stage 1 horizon)
LEVELS = [90, 95]  # Confidence levels for intervals

logger.info("NB05 — Time Series Forecasting initialized")

---
## 1. Data Loading & Exploration

The `time_series.parquet` contains monthly portfolio aggregates from 2007 to 2017 (training period). We forecast the **default rate** as our primary target.

In [ ]:
# Load pre-built time series
ts_raw = pd.read_parquet(DATA_DIR / "time_series.parquet")
print(f"Shape: {ts_raw.shape}")
print(f"Date range: {ts_raw['ds'].min().date()} to {ts_raw['ds'].max().date()}")
print(f"Months: {len(ts_raw)}")
print(f"\nColumns: {ts_raw.columns.tolist()}")
ts_raw.describe().round(4)

In [ ]:
# Prepare Nixtla-format DataFrame
df = ts_raw[["unique_id", "ds", "y"]].copy()
print(f"Target (y = default_rate): mean={df['y'].mean():.4f}, std={df['y'].std():.4f}")
print(f"Range: [{df['y'].min():.4f}, {df['y'].max():.4f}]")

# Plot the series
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Default rate
axes[0, 0].plot(ts_raw["ds"], ts_raw["y"], "b-", linewidth=1.5)
axes[0, 0].set_title("Monthly Default Rate")
axes[0, 0].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
axes[0, 0].axvspan(
    pd.Timestamp("2007-12-01"), pd.Timestamp("2009-06-01"), alpha=0.15, color="red", label="GFC"
)
axes[0, 0].legend()

# Loan count
axes[0, 1].plot(ts_raw["ds"], ts_raw["loan_count"], "g-", linewidth=1.5)
axes[0, 1].set_title("Monthly Loan Count")
axes[0, 1].set_ylabel("Count")

# Average interest rate
axes[1, 0].plot(ts_raw["ds"], ts_raw["avg_int_rate"], "r-", linewidth=1.5)
axes[1, 0].set_title("Average Interest Rate")
axes[1, 0].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

# Average DTI
axes[1, 1].plot(ts_raw["ds"], ts_raw["avg_dti"], "purple", linewidth=1.5)
axes[1, 1].set_title("Average DTI Ratio")

plt.suptitle("Lending Club Portfolio — Monthly Time Series (2007–2017)", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

---
## 2. Stationarity Analysis

Before forecasting, we test whether the default rate series is stationary using the Augmented Dickey-Fuller (ADF) test. Stationarity influences model choice: ARIMA can handle non-stationary data (via differencing), while LightGBM works on levels with lag features.

In [ ]:
from statsmodels.tsa.stattools import adfuller

# ADF test on default rate
y = df["y"].values
result = adfuller(y, maxlag=12, autolag="AIC")

print("=== Augmented Dickey-Fuller Test ===")
print(f"Test Statistic: {result[0]:.4f}")
print(f"p-value:        {result[1]:.4f}")
print(f"Lags used:      {result[2]}")
print(f"Observations:   {result[3]}")
print("Critical Values:")
for key, val in result[4].items():
    marker = " ← reject" if result[0] < val else ""
    print(f"  {key}: {val:.4f}{marker}")

if result[1] < 0.05:
    print("\n→ Series is STATIONARY (reject H0 of unit root)")
else:
    print("\n→ Series is NON-STATIONARY (cannot reject H0)")

# Seasonal decomposition
from statsmodels.tsa.seasonal import STL

# Need enough data (skip first few sparse months)
ts_for_stl = df[df["ds"] >= "2008-01-01"].set_index("ds")["y"]
stl = STL(ts_for_stl, period=12, robust=True)
res = stl.fit()

fig = res.plot()
fig.set_size_inches(14, 10)
plt.suptitle("STL Decomposition of Default Rate", fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

---
## 3. Statistical Baselines (AutoARIMA, AutoETS, AutoTheta, SeasonalNaive)

We train four statistical baselines using **statsforecast**. Each model produces point forecasts with parametric prediction intervals at 90% and 95% confidence levels.

These are our benchmarks — any ML model must beat them to justify its complexity.

In [ ]:
# Train statistical baselines
models = [
    AutoARIMA(season_length=12),
    AutoETS(season_length=12),
    AutoTheta(season_length=12),
    SeasonalNaive(season_length=12),
]

sf = StatsForecast(models=models, freq="MS", n_jobs=1)
sf.fit(df)

# Generate forecasts with intervals
fc_stats = sf.predict(h=HORIZON, level=LEVELS)
print(f"Forecast shape: {fc_stats.shape}")
print(f"Forecast columns: {fc_stats.columns.tolist()}")
print(f"\nForecast period: {fc_stats['ds'].min().date()} to {fc_stats['ds'].max().date()}")
fc_stats.head()

In [ ]:
# Visualize baseline forecasts
fig = plot_series(
    df=df,
    forecasts_df=fc_stats,
    models=["AutoARIMA", "AutoETS", "AutoTheta", "SeasonalNaive"],
    level=LEVELS,
    max_insample_length=60,
    engine="matplotlib",
)
plt.suptitle("Statistical Baseline Forecasts — Default Rate", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

---
## 4. ML Forecaster — LightGBM with Conformal Prediction Intervals

We train a LightGBM model using **mlforecast** with:
- **Lag features**: [1, 2, 3, 6, 12] months
- **Rolling statistics**: mean (3, 6, 12 months), std (6 months), EWM (α=0.3)
- **Conformal prediction intervals**: calibrated on 5 expanding backtesting windows

Unlike parametric intervals (ARIMA), conformal intervals are **distribution-free** — they make no assumption about the error distribution and provide finite-sample coverage guarantees.

In [ ]:
# ML forecaster with conformal intervals
N_WINDOWS = 5  # Backtesting windows for conformal calibration

mlf = MLForecast(
    models={
        "LightGBM": LGBMRegressor(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=4,
            num_leaves=15,
            min_child_samples=5,
            verbose=-1,
        )
    },
    freq="MS",
    lags=[1, 2, 3, 6, 12],
    lag_transforms={
        1: [
            RollingMean(window_size=3),
            RollingMean(window_size=6),
            RollingMean(window_size=12),
        ],
        3: [RollingStd(window_size=6)],
        6: [ExponentiallyWeightedMean(alpha=0.3)],
    },
    date_features=["month"],
)

# Fit with conformal prediction intervals
mlf.fit(
    df,
    prediction_intervals=PredictionIntervals(
        n_windows=N_WINDOWS,
        h=HORIZON,
        method="conformal_distribution",
    ),
)

fc_ml = mlf.predict(h=HORIZON, level=LEVELS)
print(f"ML Forecast shape: {fc_ml.shape}")
print(f"Columns: {fc_ml.columns.tolist()}")
fc_ml

In [ ]:
# Feature importance from LightGBM

lgbm_model = mlf.models_["LightGBM"]
importance = pd.DataFrame(
    {
        "feature": lgbm_model.feature_name_,
        "importance": lgbm_model.feature_importances_,
    }
).sort_values("importance", ascending=True)

fig, ax = plt.subplots(figsize=(10, 6))
ax.barh(importance["feature"], importance["importance"], color="steelblue")
ax.set_title("LightGBM Feature Importance (Lag Features)")
ax.set_xlabel("Importance (split count)")
plt.tight_layout()
plt.show()

In [ ]:
# Compare all forecasts visually
fc_combined = fc_stats.merge(fc_ml, on=["unique_id", "ds"], how="outer")

fig, ax = plt.subplots(figsize=(16, 6))

# Historical data
ax.plot(df["ds"], df["y"], "k-", linewidth=1.5, label="Actual", alpha=0.8)

# Point forecasts
models_to_plot = ["AutoARIMA", "AutoETS", "AutoTheta", "SeasonalNaive", "LightGBM"]
colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"]
for model, color in zip(models_to_plot, colors, strict=False):
    ax.plot(fc_combined["ds"], fc_combined[model], "--", color=color, linewidth=1.5, label=model)

# LightGBM conformal intervals (90%)
ax.fill_between(
    fc_ml["ds"],
    fc_ml["LightGBM-lo-90"],
    fc_ml["LightGBM-hi-90"],
    alpha=0.2,
    color="#9467bd",
    label="LightGBM 90% CI (conformal)",
)

ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.set_title("Default Rate Forecast Comparison — All Models", fontsize=14)
ax.set_xlabel("Date")
ax.set_ylabel("Default Rate")
ax.legend(loc="upper left", fontsize=9)
plt.tight_layout()
plt.show()

---
## 5. Cross-Validation Backtesting

We evaluate all models using **expanding-window cross-validation** with 3 windows of 12-month forecasts. This simulates real-world deployment where we train on all available history and forecast the next year.

The evaluation metrics are:
- **MAE**: Mean Absolute Error (interpretable in default rate units)
- **RMSE**: Root Mean Squared Error (penalizes large errors)
- **MASE**: Mean Absolute Scaled Error (normalized by naive forecast error — the standard metric in forecasting literature)

In [ ]:
# Statistical models CV
N_CV_WINDOWS = 3

sf_cv = StatsForecast(
    models=[
        AutoARIMA(season_length=12),
        AutoETS(season_length=12),
        AutoTheta(season_length=12),
        SeasonalNaive(season_length=12),
    ],
    freq="MS",
    n_jobs=1,
)

cv_stats = sf_cv.cross_validation(df=df, h=HORIZON, n_windows=N_CV_WINDOWS, level=LEVELS)
print(f"Stats CV shape: {cv_stats.shape}")
print(f"CV windows (cutoffs): {cv_stats['cutoff'].unique()}")
cv_stats.head()

In [ ]:
# ML model CV
mlf_cv = MLForecast(
    models={
        "LightGBM": LGBMRegressor(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=4,
            num_leaves=15,
            min_child_samples=5,
            verbose=-1,
        )
    },
    freq="MS",
    lags=[1, 2, 3, 6, 12],
    lag_transforms={
        1: [
            RollingMean(window_size=3),
            RollingMean(window_size=6),
            RollingMean(window_size=12),
        ],
        3: [RollingStd(window_size=6)],
        6: [ExponentiallyWeightedMean(alpha=0.3)],
    },
    date_features=["month"],
)

cv_ml = mlf_cv.cross_validation(df=df, h=HORIZON, n_windows=N_CV_WINDOWS)
print(f"ML CV shape: {cv_ml.shape}")
cv_ml.head()

In [ ]:
# Evaluate all models
stat_models = ["AutoARIMA", "AutoETS", "AutoTheta", "SeasonalNaive"]
eval_stats = evaluate(cv_stats, metrics=[mae, rmse], models=stat_models)
eval_ml = evaluate(cv_ml, metrics=[mae, rmse], models=["LightGBM"])

# Combine results
eval_all = pd.concat([eval_stats, eval_ml], axis=0, ignore_index=True)

# Aggregate across windows
eval_summary = (
    eval_all.groupby("metric")
    .apply(lambda g: g.drop(columns=["unique_id", "cutoff", "metric"]).mean())
    .round(6)
)

print("=== Cross-Validation Results (averaged across windows) ===")
print(eval_summary.to_string())

# Visual comparison
eval_melted = eval_summary.reset_index().melt(
    id_vars="metric", var_name="model", value_name="value"
)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
for i, metric_name in enumerate(["mae", "rmse"]):
    data = eval_melted[eval_melted["metric"] == metric_name].sort_values("value")
    axes[i].barh(data["model"], data["value"], color=sns.color_palette("colorblind", len(data)))
    axes[i].set_title(f"{metric_name.upper()} — Lower is Better")
    axes[i].set_xlabel(metric_name.upper())

plt.suptitle("Model Comparison — Cross-Validation Metrics", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

---
## 6. Conformal Interval Analysis

We analyze the conformal prediction intervals from the LightGBM model:
- **Coverage**: Do 90% of actual values fall within the 90% interval?
- **Width**: How wide are the intervals? Narrower = more informative.
- **Comparison**: Conformal (distribution-free) vs parametric (ARIMA) intervals.

In [ ]:
# Coverage analysis on CV results with intervals
# Re-run stats CV and ML with level to get intervals in CV
cv_stats_intervals = sf_cv.cross_validation(
    df=df, h=HORIZON, n_windows=N_CV_WINDOWS, level=[90, 95]
)


# Compute coverage for each model
def compute_coverage(cv_df, model_name, level):
    lo_col = f"{model_name}-lo-{level}"
    hi_col = f"{model_name}-hi-{level}"
    if lo_col not in cv_df.columns:
        return None
    covered = ((cv_df["y"] >= cv_df[lo_col]) & (cv_df["y"] <= cv_df[hi_col])).mean()
    width = (cv_df[hi_col] - cv_df[lo_col]).mean()
    return {"model": model_name, "level": level, "coverage": covered, "avg_width": width}


coverage_results = []
for model in stat_models:
    for level in [90, 95]:
        result = compute_coverage(cv_stats_intervals, model, level)
        if result:
            coverage_results.append(result)

# For LightGBM, re-run CV with prediction_intervals
mlf_cv2 = MLForecast(
    models={
        "LightGBM": LGBMRegressor(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=4,
            num_leaves=15,
            min_child_samples=5,
            verbose=-1,
        )
    },
    freq="MS",
    lags=[1, 2, 3, 6, 12],
    lag_transforms={
        1: [RollingMean(window_size=3), RollingMean(window_size=6), RollingMean(window_size=12)],
        3: [RollingStd(window_size=6)],
        6: [ExponentiallyWeightedMean(alpha=0.3)],
    },
    date_features=["month"],
)
mlf_cv2.fit(df, prediction_intervals=PredictionIntervals(n_windows=N_WINDOWS, h=HORIZON))
cv_ml_intervals = mlf_cv2.cross_validation(
    df=df,
    h=HORIZON,
    n_windows=N_CV_WINDOWS,
    level=[90, 95],
    prediction_intervals=PredictionIntervals(n_windows=3, h=HORIZON),
)
for level in [90, 95]:
    result = compute_coverage(cv_ml_intervals, "LightGBM", level)
    if result:
        coverage_results.append(result)

cov_df = pd.DataFrame(coverage_results)
cov_df["target"] = cov_df["level"] / 100
cov_df["coverage_gap"] = (cov_df["coverage"] - cov_df["target"]).abs()

print("=== Prediction Interval Coverage (Cross-Validation) ===")
print(cov_df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

In [ ]:
# Interval width comparison: Conformal (LightGBM) vs Parametric (AutoARIMA)
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# 90% intervals
for model, color, style in [("AutoARIMA", "#1f77b4", "--"), ("LightGBM", "#9467bd", "-")]:
    lo = f"{model}-lo-90"
    hi = f"{model}-hi-90"
    if lo in fc_combined.columns:
        axes[0].fill_between(
            fc_combined["ds"], fc_combined[lo], fc_combined[hi], alpha=0.2, color=color
        )
        axes[0].plot(
            fc_combined["ds"], fc_combined[model], style, color=color, linewidth=1.5, label=model
        )

axes[0].plot(df["ds"].tail(24), df["y"].tail(24), "k-", linewidth=1, alpha=0.5, label="History")
axes[0].set_title("90% Prediction Intervals — Parametric vs Conformal")
axes[0].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
axes[0].legend()

# Interval width by horizon step
if "LightGBM-lo-90" in fc_ml.columns:
    ml_widths = fc_ml["LightGBM-hi-90"] - fc_ml["LightGBM-lo-90"]
    arima_widths = fc_stats["AutoARIMA-hi-90"] - fc_stats["AutoARIMA-lo-90"]
    horizon_steps = range(1, HORIZON + 1)

    axes[1].plot(
        horizon_steps, arima_widths.values, "o--", label="AutoARIMA (parametric)", color="#1f77b4"
    )
    axes[1].plot(
        horizon_steps, ml_widths.values, "s-", label="LightGBM (conformal)", color="#9467bd"
    )
    axes[1].set_xlabel("Forecast Horizon (months)")
    axes[1].set_ylabel("Interval Width")
    axes[1].set_title("90% Interval Width by Horizon Step")
    axes[1].legend()

plt.tight_layout()
plt.show()

# Print width stats
print(f"\nAutoARIMA 90% avg width: {arima_widths.mean():.4f}")
print(f"LightGBM  90% avg width: {ml_widths.mean():.4f}")

---
## 7. Multi-Target Forecasting — Default Rate + Loan Volume

Besides default rate, we also forecast **loan count** to demonstrate the pipeline's versatility. Both series are important for credit risk management:
- Default rate → provisioning and stress testing
- Loan volume → capacity planning and concentration risk

In [ ]:
# Build multi-series DataFrame
df_volume = ts_raw[["ds"]].copy()
df_volume["unique_id"] = "loan_count"
df_volume["y"] = ts_raw["loan_count"].astype(float)

df_multi = pd.concat([df, df_volume], ignore_index=True)
print(f"Multi-series shape: {df_multi.shape}")
print(f"Series: {df_multi['unique_id'].unique()}")

# Forecast both with statsforecast
sf_multi = StatsForecast(
    models=[AutoARIMA(season_length=12), AutoETS(season_length=12)],
    freq="MS",
    n_jobs=1,
)
fc_multi = sf_multi.forecast(h=HORIZON, df=df_multi, level=[90])

# Plot
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

for i, series_id in enumerate(["portfolio", "loan_count"]):
    hist = df_multi[df_multi["unique_id"] == series_id]
    fcast = fc_multi[fc_multi["unique_id"] == series_id]

    axes[i].plot(hist["ds"], hist["y"], "k-", linewidth=1.5, label="Actual")
    for model, color in [("AutoARIMA", "#1f77b4"), ("AutoETS", "#ff7f0e")]:
        axes[i].plot(fcast["ds"], fcast[model], "--", color=color, label=model)
        lo, hi = f"{model}-lo-90", f"{model}-hi-90"
        axes[i].fill_between(fcast["ds"], fcast[lo], fcast[hi], alpha=0.15, color=color)

    title = "Default Rate" if series_id == "portfolio" else "Loan Count"
    axes[i].set_title(f"{title} — 12-Month Forecast")
    if series_id == "portfolio":
        axes[i].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    axes[i].legend(fontsize=9)

plt.tight_layout()
plt.show()

---
## 8. Grade-Level Forecasting & Hierarchical Reconciliation

We build grade-level time series (A-G), each with its own default rate dynamics:

- Grade A: ~5.5% default rate (low risk, stable)
- Grade G: ~42% default rate (high risk, volatile)

We demonstrate **hierarchical reconciliation** using the  function from hierarchicalforecast. The summation matrix S encodes that portfolio = sum of grades.

> **Note**: Default rates are proportions and not directly summable across groups. The hierarchy treats each grade's rate as an independent "unit" and the portfolio as their sum. For production use, one would forecast default *counts* and reconcile those (since counts are additive), then recompute rates. Here we show the technique applied to rate series as a pedagogical demonstration.

In [ ]:
# Build grade-level time series from loan_master
from hierarchicalforecast.utils import aggregate

lm = pd.read_parquet(DATA_DIR / "loan_master.parquet")

# Monthly aggregation by grade
grade_monthly = (
    lm.groupby([pd.Grouper(key="issue_d", freq="MS"), "grade"])
    .agg(default_rate=("default_flag", "mean"))
    .reset_index()
)
grade_monthly.columns = ["ds", "grade", "y"]
grade_monthly["total"] = "portfolio"

# Build hierarchy: total -> grade
Y_df, S_df, tags = aggregate(
    df=grade_monthly,
    spec=[["total"], ["total", "grade"]],
)

print(f"Hierarchical series: {Y_df['unique_id'].nunique()} ({Y_df['unique_id'].unique().tolist()})")
print(f"Observations: {Y_df.shape[0]}")
print(f"S_df shape: {S_df.shape}")

# Show stats per series
avg_by_id = Y_df.groupby("unique_id")["y"].agg(["mean", "count"]).round(4)
print("Stats by series:")
print(avg_by_id.to_string())
print()
print("Note: portfolio y = sum of grade rates (not a true avg default rate)")
print(f"True portfolio avg default rate: {grade_monthly['y'].mean():.4f}")

In [ ]:
# Forecast all series with AutoARIMA
sf_hier = StatsForecast(
    models=[AutoARIMA(season_length=12), AutoETS(season_length=12)],
    freq="MS",
    n_jobs=1,
)
Y_hat_df = sf_hier.forecast(h=HORIZON, df=Y_df, level=[90])

print(f"Base forecasts shape: {Y_hat_df.shape}")
print(f"Series forecasted: {Y_hat_df['unique_id'].unique().tolist()}")
Y_hat_df.head(16)

In [ ]:
# Reconcile with MinTrace (OLS)
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import BottomUp, MinTrace

hrec = HierarchicalReconciliation(
    reconcilers=[
        BottomUp(),
        MinTrace(method="ols"),
    ]
)

Y_reconciled = hrec.reconcile(
    Y_hat_df=Y_hat_df,
    tags=tags,
    S_df=S_df,
    Y_df=Y_df,
)

print(f"Reconciled shape: {Y_reconciled.shape}")
print(f"Reconciled columns: {Y_reconciled.columns.tolist()[:10]}...")

# Compare base vs reconciled for portfolio total
portfolio_base = Y_hat_df[Y_hat_df["unique_id"] == "portfolio"]
portfolio_recon = Y_reconciled[Y_reconciled["unique_id"] == "portfolio"]

print("\nPortfolio forecast comparison (AutoARIMA):")
print(f"  Base:       {portfolio_base['AutoARIMA'].mean():.4f}")
if "AutoARIMA/BottomUp" in Y_reconciled.columns:
    print(f"  BottomUp:   {portfolio_recon['AutoARIMA/BottomUp'].mean():.4f}")
if "AutoARIMA/MinTrace_method-ols" in Y_reconciled.columns:
    print(f"  MinTrace:   {portfolio_recon['AutoARIMA/MinTrace_method-ols'].mean():.4f}")

In [ ]:
# Visualize grade-level forecasts
grades = sorted([uid for uid in Y_hat_df["unique_id"].unique() if "/" in uid])

fig, axes = plt.subplots(2, 4, figsize=(20, 8))
axes = axes.flatten()

for i, grade_id in enumerate(["portfolio"] + grades):
    if i >= len(axes):
        break
    ax = axes[i]

    # History
    hist = Y_df[Y_df["unique_id"] == grade_id]
    fcast = Y_hat_df[Y_hat_df["unique_id"] == grade_id]

    ax.plot(hist["ds"], hist["y"], "k-", linewidth=1, alpha=0.7)
    ax.plot(fcast["ds"], fcast["AutoARIMA"], "b--", linewidth=1.5)
    ax.fill_between(
        fcast["ds"], fcast["AutoARIMA-lo-90"], fcast["AutoARIMA-hi-90"], alpha=0.2, color="blue"
    )
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

    label = grade_id.split("/")[-1] if "/" in grade_id else "Portfolio (Total)"
    avg_def = hist["y"].mean()
    ax.set_title(f"{label} (avg={avg_def:.1%})", fontsize=11)

# Hide unused axes
for j in range(i + 1, len(axes)):
    axes[j].set_visible(False)

plt.suptitle("Grade-Level Default Rate Forecasts (AutoARIMA + 90% CI)", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

---
## 9. IFRS 9 & Stress Testing Implications

The time series forecasts with conformal intervals directly support regulatory requirements:

### Forward-Looking ECL (IFRS 9 para. 5.5.17)
IFRS 9 requires incorporating **reasonable and supportable forecasts** into ECL estimates. Our forecasted default rates with intervals provide:
- **Baseline scenario**: Point forecast
- **Optimistic scenario**: Lower bound of 90% interval
- **Adverse scenario**: Upper bound of 95% interval

### Data-Driven Stress Testing
The upper bound of conformal intervals serves as a **model-driven stress scenario** — more principled than ad-hoc macro assumptions, with mathematical coverage guarantees.

In [ ]:
# IFRS 9 Scenario Analysis
print("=== IFRS 9 Forward-Looking Default Rate Scenarios ===")
print(f"Forecast horizon: {HORIZON} months (Stage 1 horizon)\n")

# LightGBM conformal forecasts
scenarios = pd.DataFrame(
    {
        "month": fc_ml["ds"],
        "point_forecast": fc_ml["LightGBM"],
        "optimistic_90": fc_ml["LightGBM-lo-90"],
        "adverse_90": fc_ml["LightGBM-hi-90"],
        "optimistic_95": fc_ml["LightGBM-lo-95"],
        "adverse_95": fc_ml["LightGBM-hi-95"],
    }
)

print("Monthly Scenarios:")
print(scenarios.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

# Summary
print("\n=== 12-Month Average Default Rate by Scenario ===")
print(f"  Historical avg (last 12m): {df['y'].tail(12).mean():.4f}")
print(f"  Optimistic (lo-95):        {scenarios['optimistic_95'].mean():.4f}")
print(f"  Optimistic (lo-90):        {scenarios['optimistic_90'].mean():.4f}")
print(f"  Baseline (point):          {scenarios['point_forecast'].mean():.4f}")
print(f"  Adverse (hi-90):           {scenarios['adverse_90'].mean():.4f}")
print(f"  Adverse (hi-95):           {scenarios['adverse_95'].mean():.4f}")

# Stress impact
stress_ratio = scenarios["adverse_95"].mean() / scenarios["point_forecast"].mean()
print(f"\n  Stress multiplier (95% / baseline): {stress_ratio:.2f}x")
print(f"  → Provisions under stress scenario should be {stress_ratio:.2f}x the baseline")

In [ ]:
# Scenario fan chart
fig, ax = plt.subplots(figsize=(14, 6))

# History
ax.plot(df["ds"].tail(36), df["y"].tail(36), "k-", linewidth=2, label="Historical")

# Forecast scenarios
ax.plot(fc_ml["ds"], fc_ml["LightGBM"], "b-", linewidth=2, label="Baseline (point)")
ax.fill_between(
    fc_ml["ds"],
    fc_ml["LightGBM-lo-90"],
    fc_ml["LightGBM-hi-90"],
    alpha=0.25,
    color="blue",
    label="90% Conformal Interval",
)
ax.fill_between(
    fc_ml["ds"],
    fc_ml["LightGBM-lo-95"],
    fc_ml["LightGBM-hi-95"],
    alpha=0.12,
    color="blue",
    label="95% Conformal Interval",
)

# Annotations
ax.axhline(
    y=scenarios["adverse_95"].mean(),
    color="red",
    linestyle=":",
    alpha=0.6,
    label=f"Stress scenario avg ({scenarios['adverse_95'].mean():.1%})",
)

ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.set_title("IFRS 9 Forward-Looking Scenarios — Conformal Prediction Intervals", fontsize=14)
ax.set_xlabel("Date")
ax.set_ylabel("Default Rate")
ax.legend(loc="upper left")
plt.tight_layout()
plt.show()

---
## 10. Sensitivity Analysis — Horizon and Confidence Level

We examine how forecast quality degrades with horizon length and how interval width changes with confidence level.

In [ ]:
# Sensitivity: confidence level vs interval width
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: interval width vs confidence level (from final forecast)
levels_test = [50, 70, 80, 90, 95, 99]
widths_by_level = []

for lev in levels_test:
    fc_test = mlf.predict(h=HORIZON, level=[lev])
    lo = f"LightGBM-lo-{lev}"
    hi = f"LightGBM-hi-{lev}"
    w = (fc_test[hi] - fc_test[lo]).mean()
    widths_by_level.append({"level": lev, "avg_width": w})

width_df = pd.DataFrame(widths_by_level)
axes[0].plot(width_df["level"], width_df["avg_width"], "bo-", linewidth=2, markersize=8)
axes[0].set_xlabel("Confidence Level (%)")
axes[0].set_ylabel("Average Interval Width")
axes[0].set_title("Interval Width vs Confidence Level")
axes[0].set_xticks(levels_test)

# Right: coverage vs horizon step (from CV)
if "LightGBM-lo-90" in cv_ml_intervals.columns:
    cv_ml_intervals["step"] = cv_ml_intervals.groupby(["unique_id", "cutoff"]).cumcount() + 1
    coverage_by_step = []
    for step in range(1, HORIZON + 1):
        mask = cv_ml_intervals["step"] == step
        if mask.sum() > 0:
            subset = cv_ml_intervals[mask]
            covered = (
                (subset["y"] >= subset["LightGBM-lo-90"])
                & (subset["y"] <= subset["LightGBM-hi-90"])
            ).mean()
            coverage_by_step.append({"step": step, "coverage": covered})

    cov_step_df = pd.DataFrame(coverage_by_step)
    axes[1].bar(cov_step_df["step"], cov_step_df["coverage"], color="steelblue", alpha=0.7)
    axes[1].axhline(y=0.90, color="red", linestyle="--", label="Target 90%")
    axes[1].set_xlabel("Forecast Horizon Step (months)")
    axes[1].set_ylabel("Empirical Coverage")
    axes[1].set_title("90% Coverage by Horizon Step")
    axes[1].legend()
    axes[1].set_ylim(0, 1.1)
else:
    axes[1].text(
        0.5,
        0.5,
        "Conformal intervals not available in CV",
        transform=axes[1].transAxes,
        ha="center",
    )

plt.tight_layout()
plt.show()

---
## 11. Save Artifacts

Save forecasts, model objects, and evaluation metrics for downstream notebooks.

In [ ]:
import pickle

# Save forecasts
fc_combined.to_parquet(DATA_DIR / "ts_forecasts.parquet", index=False)
logger.info(f"Saved ts_forecasts.parquet: {fc_combined.shape}")

# Save ML model
with open(MODEL_DIR / "ts_mlforecast.pkl", "wb") as f:
    pickle.dump(mlf, f)
logger.info("Saved ts_mlforecast.pkl")

# Save hierarchical forecasts
Y_hat_df.to_parquet(DATA_DIR / "ts_hierarchical_forecasts.parquet", index=False)
logger.info(f"Saved ts_hierarchical_forecasts.parquet: {Y_hat_df.shape}")

# Save CV results
cv_stats_intervals.to_parquet(DATA_DIR / "ts_cv_stats.parquet", index=False)
logger.info(f"Saved ts_cv_stats.parquet: {cv_stats_intervals.shape}")

# Save IFRS9 scenarios
scenarios.to_parquet(DATA_DIR / "ts_ifrs9_scenarios.parquet", index=False)
logger.info(f"Saved ts_ifrs9_scenarios.parquet: {scenarios.shape}")

print("\n✓ All time series artifacts saved")

---
## Summary

### Models Trained
| Model | Type | Prediction Intervals |
|-------|------|---------------------|
| AutoARIMA | Statistical | Parametric (Gaussian) |
| AutoETS | Statistical | Parametric (Gaussian) |
| AutoTheta | Statistical | Parametric |
| SeasonalNaive | Baseline | Parametric |
| LightGBM | ML (mlforecast) | **Conformal** (distribution-free) |

### Key Results
- **Cross-validation metrics** compared across all models (MAE, RMSE)
- **Conformal intervals** from LightGBM provide distribution-free coverage guarantees
- **Grade-level hierarchy** forecasted and reconciled (7 grades + portfolio total)
- **IFRS 9 scenarios** derived from conformal interval bounds

### Artifacts Saved
- `data/processed/ts_forecasts.parquet` — Combined forecasts (all models)
- `data/processed/ts_hierarchical_forecasts.parquet` — Grade-level forecasts
- `data/processed/ts_ifrs9_scenarios.parquet` — IFRS 9 scenario table
- `data/processed/ts_cv_stats.parquet` — CV results with intervals
- `models/ts_mlforecast.pkl` — Fitted LightGBM forecaster

### Next Steps
1. **NB06**: Survival Analysis — lifetime PD curves for IFRS 9 Stage 2
2. **NB08**: Portfolio Optimization — use both loan-level and portfolio-level intervals as uncertainty sets

## Final Conclusions: Time Series Forecasting

### Key Findings
- Statistical and ML forecasters provide consistent short-horizon default-rate projections.
- Forecast outputs support scenario framing rather than single-point planning.
- Interval forecasts are necessary for governance under uncertain cycle transitions.

### Financial Risk Interpretation
- Portfolio-level default dynamics drive provisioning and capital planning cadence.
- Forecast uncertainty should be propagated into strategic risk appetite and stress interpretation.
- Trend and seasonality shifts require continuous monitoring, not one-off model deployment.

### Contribution to End-to-End Pipeline
- Produces forward-looking default-rate views that complement loan-level PD models.
- Supports macro-level planning context for IFRS9 and portfolio strategy.
- Adds a temporal risk lens that the cross-sectional PD model alone cannot provide.